# Similarity Search: Treatment-Control Matching for DRC Settlements

This notebook performs cosine-similarity matching to construct a control group for causal inference analysis of settlements in North and South Kivu.

---
## 1. Setup & Imports

In [66]:
import geopandas as gpd
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

---
## 2. Load Data

Read the settlement-level GeoDataFrame. Each row is a settlement with annual columns for area, population, road length, electrification, and the Atlas Wealth Index (AWI).

In [67]:
# NOTE: Update this path to point to your local parquet file
PARQUET_PATH = '/Users/phoebely/Documents/Projects/Wellspring/version2_gdf.parquet'

drc_features = gpd.read_parquet(PARQUET_PATH)
print(f"Loaded {len(drc_features):,} settlements with {drc_features.shape[1]} columns")
drc_features.head()

Loaded 138,803 settlements with 51 columns


,UID,geometry,area (sq km)_2016,pop_2016_sum,road_length_m_2016,elec_2016_area_km2,awi_2016_mean,area (sq km)_2017,pop_2017_sum,road_length_m_2017,...,awi_2023_mean,area (sq km)_2024,pop_2024_sum,road_length_m_2024,elec_2024_area_km2,awi_2024_mean,area (sq km)_2025,pop_2025_sum,road_length_m_2025,elec_2025_area_km2
0,0.0,"POLYGON ((3019093.912 -422065.195, 3019095.986...",0.000000,0.0,0.0,0.0,NaN,0.000000,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,NaN,0.085749,0.0,0.0,0.0
1,1.0,"POLYGON ((3018791.314 -421751.604, 3018796.228...",0.001116,0.0,0.0,0.0,-0.664234,0.000202,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,NaN,0.056674,0.0,0.0,0.0
2,2.0,"POLYGON ((3018368.075 -420404.331, 3018370.575...",0.000000,0.0,0.0,0.0,NaN,0.000000,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,NaN,0.014839,0.0,0.0,0.0
3,3.0,"POLYGON ((3017862.729 -419762.781, 3017866.689...",0.000000,0.0,0.0,0.0,NaN,0.000000,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,NaN,0.019787,0.0,0.0,0.0
4,4.0,"POLYGON ((3017622.124 -419681.306, 3017624.255...",0.000000,0.0,0.0,0.0,NaN,0.000000,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,NaN,0.018818,0.0,0.0,0.0


---
## 3. Assign Treatment Group

Currently a random selection of 100 settlements are selected as the "treated" group. User will need to assign treatment settlements based on context.

In [ ]:
N_TREATED = 100
RANDOM_SEED = 123

awi_cols = [c for c in drc_features.columns if 'awi' in c]
has_awi = drc_features.dropna(subset=awi_cols) #dropping where AWI is null
treated_idx = has_awi.sample(n=N_TREATED, random_state=RANDOM_SEED).index
drc_features['treated'] = drc_features.index.isin(treated_idx)

print(f"Treated: {drc_features['treated'].sum():,}  "
      f"Control pool: {(~drc_features['treated']).sum():,}")

Treated: 100  Control pool: 138,703


---
## 4. Standardize Matching Features

We must standardize our data to ensure all features contribute equally to the score. Missing values are filled with 0 before scaling.

In [86]:
# Select all numeric columns except UID and treated flag
EXCLUDE_COLS = ['UID', 'treated', 'geometry']
MATCH_FEATURES = [
    col for col in drc_features.select_dtypes(include='number').columns
    if col not in EXCLUDE_COLS
]

scaler = StandardScaler()
standardized_cols = [f'{c}_standardized' for c in MATCH_FEATURES]
drc_features[standardized_cols] = scaler.fit_transform(
    drc_features[MATCH_FEATURES].fillna(0)
)


---
## 5. Split Treatment and Control Pools

In [87]:
treated = drc_features[drc_features['treated']].copy()
candidate = drc_features[~drc_features['treated']].copy()

print(f"Treated: {len(treated):,}  |  Candidate controls: {len(candidate):,}")

Treated: 100  |  Candidate controls: 138,703


---
## 6. Cosine Similarity Search

For each treated settlement, find the **top 10** most similar candidates by cosine similarity on the standardized baseline features.

In [88]:
TOP_K = 10

treat_matrix = treated[standardized_cols].fillna(0).values
cand_matrix = candidate[standardized_cols].fillna(0).values
cand_uids = candidate['UID'].values

sims = cosine_similarity(cand_matrix, treat_matrix)

matched_uids = []
for col_idx in range(sims.shape[1]):
    top_idx = np.argpartition(sims[:, col_idx], -TOP_K)[-TOP_K:]
    matched_uids.extend(cand_uids[top_idx].tolist())

print(f"Total matches: {len(matched_uids):,}  |  Unique UIDs: {len(set(matched_uids)):,}")

Total matches: 1,000  |  Unique UIDs: 976


### Build Control Group

Flag matched candidates and extract the donor pool.

In [89]:
candidate['control_group'] = candidate['UID'].isin(matched_uids).astype(int)
donor_pool = candidate[candidate['control_group'] == 1].copy()

print(f"Donor pool: {len(donor_pool):,} settlements")

Donor pool: 976 settlements


---
## 7. Compute Geographic Coordinates

Reproject to WGS 84 (EPSG:4326) and extract centroid latitude/longitude. Centroids are computed before reprojecting to avoid the geographic-CRS warning.

In [90]:
def add_lat_long(gdf):
    centroids = gdf.geometry.centroid
    centroids_4326 = centroids.to_crs(4326)
    gdf = gdf.to_crs(4326)
    gdf['long'] = centroids_4326.x
    gdf['lat'] = centroids_4326.y
    return gdf

treated = add_lat_long(treated)
donor_pool = add_lat_long(donor_pool)

---
## 8. Build Long Panel for Causal Inference

Reshape the AWI outcome variable from wide to long format and add treatment indicators.

### 8a. Define AWI year columns and treatment timing

In [91]:
# AWI columns to include in the panel
AWI_COLS = [
    'awi_2016_mean', 'awi_2017_mean', 'awi_2018_mean',
    'awi_2019_mean', 'awi_2020_mean', 'awi_2021_mean',
    'awi_2022_mean', 'awi_2023_mean', 'awi_2024_mean'
]

# Map column names to numeric years
COL_TO_YEAR = {col: int(col.split('_')[1]) for col in AWI_COLS}

# Treatment timing for Rutshuru: treatment starts in 2020
TREAT_YEAR_MAP = {
    2016: 0, 2017: 0, 2018: 0,
    2019: 0, 2020: 1, 2021: 1,
    2022: 1, 2023: 1, 2024: 1,
}

### 8b. Melt treatment and control groups to long format

In [92]:
def melt_to_panel(gdf, awi_cols, col_to_year, is_treated):
    subset = gdf[['UID'] + awi_cols].copy()
    melted = pd.melt(
        subset, id_vars='UID', value_vars=awi_cols,
        var_name='year_temp', value_name='awi',
    )
    melted['year'] = melted['year_temp'].map(col_to_year)
    melted['treated_unit'] = int(is_treated)
    melted['treat_year'] = melted['year'].map(TREAT_YEAR_MAP)
    melted['treated_id'] = melted['treat_year'] if is_treated else 0
    melted.drop(columns='year_temp', inplace=True)
    return melted

treat_long = melt_to_panel(treated, AWI_COLS, COL_TO_YEAR, is_treated=True)
control_long = melt_to_panel(donor_pool, AWI_COLS, COL_TO_YEAR, is_treated=False)

print(f"Treatment panel rows: {len(treat_long):,}")
print(f"Control panel rows:   {len(control_long):,}")

Treatment panel rows: 900
Control panel rows:   8,784


### 8c. Merge additional columns and finalize

In [93]:
# Additional columns: coordinates and population
add_cols_treat = treated[['UID', 'lat', 'long', 'pop_2025_sum']].rename(columns={'pop_2025_sum': 'n'})
add_cols_control = donor_pool[['UID', 'lat', 'long', 'pop_2025_sum']].rename(columns={'pop_2025_sum': 'n'})

treat_long = treat_long.merge(add_cols_treat, on='UID')
control_long = control_long.merge(add_cols_control, on='UID')

# Create a unique row identifier
treat_long['UID_year'] = treat_long['UID'].astype(str) + '_' + treat_long['year'].astype(str)
control_long['UID_year'] = control_long['UID'].astype(str) + '_' + control_long['year'].astype(str)

# Select and order final columns
FINAL_COLS = ['UID_year', 'UID', 'lat', 'long', 'n', 'year',
              'treated_unit', 'treat_year', 'treated_id', 'awi']

treat_final = treat_long[FINAL_COLS]
control_final = control_long[FINAL_COLS]

---
## 9. Inspect Results

In [94]:
print("=== Treatment Panel ===")
display(treat_final.head())
print(f"\n=== Control Panel ===")
display(control_final.head())

=== Treatment Panel ===


,UID_year,UID,lat,long,n,year,treated_unit,treat_year,treated_id,awi
0,209.0_2016,209.0,-2.987962,27.071638,184.756454,2016,1,0,0,-0.595508
1,2118.0_2016,2118.0,-3.250709,27.172654,24.849018,2016,1,0,0,-0.571595
2,2325.0_2016,2325.0,-3.047287,27.153911,6.718816,2016,1,0,0,-0.671216
3,3437.0_2016,3437.0,-3.303776,27.186328,0.000000,2016,1,0,0,-0.586995
4,5481.0_2016,5481.0,-2.677719,27.320249,203.922623,2016,1,0,0,-0.608816



=== Control Panel ===


,UID_year,UID,lat,long,n,year,treated_unit,treat_year,treated_id,awi
0,137.0_2016,137.0,-3.089189,27.121818,18.481844,2016,0,0,0,-0.617462
1,550.0_2016,550.0,-2.815293,27.121232,4.806621,2016,0,0,0,-0.682457
2,980.0_2016,980.0,-2.546193,27.079477,0.000000,2016,0,0,0,-0.714408
3,992.0_2016,992.0,-2.535240,27.056891,68.143600,2016,0,0,0,-0.706401
4,2027.0_2016,2027.0,-3.330232,27.149350,11.152165,2016,0,0,0,-0.621134


In [95]:
treat_final.to_csv('treat_final_v2.csv', index=False)
control_final.to_csv('control_final_v2.csv', index=False)